In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd
import seaborn as sns
import random

In [3]:

test_features = pd.read_csv('test_features.csv', names=('id','dna'),
                          na_values='' , comment='\t',
                          sep=',', skipinitialspace=True)
train_features = pd.read_csv('train_features.csv', names=('id','dna'),
                          na_values='' , comment='\t',
                          sep=',', skipinitialspace=True)

train_labels = pd.read_csv('train_labels.csv', names=('id','labels'),
                          na_values='' , comment='\t',
                          sep=',', skipinitialspace=True)

In [5]:
test_features.tail()

,id,dna
8302,8302,----------------------------------------------...
8303,8303,TACATTATATTTTTTATTTGGAACTTGAGCTGGAATAGTTGGTACT...
8304,8304,AACATTATATTTTATTTTTGGAGCATGATCAGGAATAGTTGGAACA...
8305,8305,TACTTTATATTTTATTTTCGGAGCTTGAGCTGGTATAGTAGGAACG...
8306,8306,GACTTTGTATTTTCTATTTGGAATATGAGCTGGAATAATTGGAACT...


In [6]:
train_features.tail()

,id,dna
12902,12902,TACTCTGTATTTTCTATTTGGAGTATGATCAGGAATAGTAGGAACA...
12903,12903,AACATTATATTTTATCTTTGGGGCCTGATCAGGAATAGTAGGAACT...
12904,12904,AACCTTATATTTCCTATTCGGAGCATGAGCCGGAATATTAGGAACA...
12905,12905,----------------------------------------------...
12906,12906,AACATTATATTTTATTTTTGGGGCTTGAGCTGGAATAGTTGGAACT...


In [7]:
train_labels.tail()

,id,labels
12902,12902,400
12903,12903,171
12904,12904,329
12905,12905,1148
12906,12906,406


In [8]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
from torchsummary import summary
print(net)
summary(net,(3,32,32))

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_labels, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# predict on test data
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))